In [32]:
import torch
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

In [33]:
# In our last example, we modeld a NN, that took in one character (out of 27 posibile characters) to predict what the probabilyt was for
# next character to be one of the 27 possiblebites.  To do this, we took the input character, an index from 0 to 26, mapped it to a one 
# hot, and used that as the vector into the network.  
Image(url="nn-lm-1.png", width=1100)

In [34]:
# We simplifed this to 1 layer as below.
Image(url="nn-lm-2.png", width=750)


In [35]:
# Now if we want to predict the next output from 3 past inputs instead of one, we get the following
Image(url="nn-lm-3.png", width=1100)

In [36]:
# Note that the input is the conatactintion of the 3 past inputs.  With one-hot encoding each input was a 1x27 vector.  Now with 3 inputs, 
# conactenate theim and get a 1x82 vector.  That makes the first hidden layer quite large.  Also with 3 past vaulyes the combinatoral poisbilitys 
# are 27x27x27. Gettting probabilitys for each 27x27x27 possibiilty wil be difficult, as there are so many indivial possibilityes, the probabilyt 
# of any one particulr combo is going to be low.

# So instead we add an embedding layer.  the Embnedong layer is just a matirix, whose weights get trainged along with all the other wietts, and 
# we multiply the high dimnetsial input vectors by the emdeding matrix to map them into lower dimensions.  So a 1x27 one-hot vector when mutliplied
# by a 27x3 embedding matrix will map each 1x27 vector (27 dimensions) into a 1x3 vector (3 dimensoins).   Because the wigths of this matrix 
# get trained to help minimize the loss functions, the eventaul values of the wieghts of the emdiingin matrix end up having meaning in the 
# problem space we are trying to solve.  For example, since we are mapping to lower diminssine, and tranpsoition of voweks in a name is fine for
# new name in terms of loss function, the transofrm provided my the matrix multiplcation that maps to lower dimension, would work out to make differnt 
# vowles, each a very diffent input one-hot vector map to the same or similar emdedded lower dimensial vector since voels are interchangeble in a 
# radimly constructed name.

# For 3 inputs, we one-hot each input, pass them through the embeeding matrix (vect*matrix) and take the reduced dimson resutls and conate them into
# a now smaller input to the rest of the netowrk.  See below.
Image(url="nn-lm-4.png", width=1100)

In [37]:
# Note that the vector*matrix multiplication is differentable (as all multipcation and add is) so we can do backprop to minimize the
# the wieghts just fine.  Also note that since the input vectros to the vect*matrix are one-hots, matrix multiply with one hot vectors 
# is just the same as selcting the row of the matix that corresponds to the index used to create the one hot vector.  So if index 3 was
# used to create one hot vector of v=[0, 0, 0, 1, 0, 0, ..., 0] and if we have emedding marix m as a 27x3 matrix, then v*m is just 
# the 3rd row of matirx m, or m[3].

In [44]:
# We will implemt a network from a paper by Bengio.  It is exactly same as the netowrk we propsed above, it 
# just has only one hidden layer instead of two.  See below. 
Image(url="bengio.png", width=700)

In [39]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()


In [40]:
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [41]:
len(words)

32033

In [42]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [43]:
# build the dataset
X = []
Y = []
block_size = 3 # context length: how many characters do we take to predict the next one?

for w in words:
  
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)
X.shape, X.dtype, Y.shape, Y.dtype

W = torch.randn(3, 1)
W

tensor([[ 0.5481],
        [-1.2230],
        [ 1.5021]])